## Imports

In [6]:
import os
import csv
import xml.etree.ElementTree as ET
import pandas as pd
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.io import read_image
import matplotlib.pyplot as plt
from matplotlib import pyplot
import numpy as np
import torch.nn.functional as F
from torchvision.utils import draw_bounding_boxes
from torchvision.transforms import ColorJitter
from torchvision.transforms import GaussianBlur
from matplotlib.patches import Rectangle
import torchvision.transforms.functional as TF
# from torchvision.transforms import functional as F
import matplotlib.patches as patches
import glob
import pickle
import random

import networkx as nx
import itertools

from math import sqrt as sqrt
from itertools import product as product
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import torch.nn.init as init

import torchvision.transforms as transforms
import torchvision.models as models
import torch.backends.cudnn as cudn


## Pickle

In [4]:
pickle_path = "C:/Users/Gaura/OneDrive/Desktop/GAURAV/Unimleb/COMP90051/gt/02/Statisitcal_Machine_Learning_Project/"       ## Change this location
# pickle_path = "C:/Users/Gaura/OneDrive/Desktop/GAURAV/Unimleb/COMP90051/gt/02/Statisitcal_Machine_Learning_Project/"       ## Change this location

pickle_file_path = os.path.join(pickle_path, 'WSR_train_data.pkl')

# Load the pickle file
with open(pickle_file_path, 'rb') as file:
    wsr_data = pickle.load(file)

filenames = wsr_data['file_names']  # List of filenames
resized_images = wsr_data['images']  # Corresponding image tensors (resized)
object_location = wsr_data['object_locations'] # Corresponding to new bb co-ordinates


pickle_path = "C:/Users/Gaura/OneDrive/Desktop/GAURAV/Unimleb/COMP90051/gt/02/Statisitcal_Machine_Learning_Project/"       ## Change this location
pickle_file_path = os.path.join(pickle_path, 'train_dataset.pkl')

# Load the pickle file
with open(pickle_file_path, 'rb') as file:
    all_data = pickle.load(file)

# filenames = data['file_names']  # List of filenames
# resized_images = data['images']  # Corresponding image tensors (resized)
# object_location = data['object_locations'] # Corresponding to new bb co-ordinates

## Functions

In [2]:
torch_to_np = lambda tse : tse.permute(1, 2, 0).numpy()
np_to_torch = lambda npa : torch.from_numpy(npa).permute(2, 0, 1)


def image_resizer(img, new_dim):
    new_h, new_w = new_dim
    resize_transform = transforms.Resize((new_h, new_w))
    image_resized = resize_transform(transforms.ToPILImage()(img))  # Convert to PIL image and resize
    return transforms.ToTensor()(image_resized)

def object_location_resizer(objs, old_dim = (584, 688),  new_dim = (512, 512)):
    old_height, old_width = old_dim
    new_height, new_width = new_dim
    height_ratio = new_height/old_height
    width_ratio = new_width/old_width

    ratioLst = [width_ratio, height_ratio, width_ratio, height_ratio]
    new_boxes = []
    for box in objs:
        box = box.tolist()
        box = [abs(int(a * b)) for a, b in zip(box, ratioLst)] 
        new_boxes.append(box)
    return torch.tensor(new_boxes)

def show_image(pic, obj=False, class_code = False, convert=False, font_size=12):
    if isinstance(pic, torch.Tensor):
        pic = torch_to_np(pic)
    if convert:
        pic = image.imread(pic)
    
    pyplot.imshow(pic)
    if torch.is_tensor(obj):
        code_to_obj = dict(zip([q for q in range(5)], ["gun", "knife", "plier", "scissor", "wrench"]))
        # print(code_to_obj)
        temp = obj.tolist()
        obj = {}
        for i, bb in enumerate(temp):
            # obj[i] = [int(class_code[i]), bb]
            obj[i] = [code_to_obj[int(class_code[i])], bb]
    print(f"dtype : {pic.dtype}")
    print(f"Shape : {pic.shape}")
    print(f"Aspect Ratio (h/w) : {round(pic.shape[0] / pic.shape[1], 2)}")
    if obj:
        labels = []
        bound_box = []
        for k, v in obj.items():
            labels.append(v[0])
            bound_box.append(v[1])
        
        # Draw bounding boxes and labels
        for i, bbox in enumerate(bound_box):
            x_min, y_min, x_max, y_max = bbox
            width, height = x_max - x_min, y_max - y_min
            
            # Add a rectangle patch for each bounding box
            rect = Rectangle((x_min, y_min), width, height, linewidth=3, edgecolor='red', facecolor='none')
            pyplot.gca().add_patch(rect)
            
            # Add text label above the bounding box
            pyplot.text(x_min, y_min - 5, labels[i], color='white', fontsize=font_size, 
                        bbox=dict(facecolor='red', edgecolor='red', pad=2.0))            

sorted_join = lambda st : "".join(sorted(list(set("".join(sorted(st))))))

get_area = lambda arbb : (arbb[2]-arbb[0])*(arbb[3]-arbb[1])

def calculate_intersection_area(box1, box2, get_intersection_box = False):
    # Box format: [x_min, y_min, x_max, y_max]
    x_min1, y_min1, x_max1, y_max1 = box1
    x_min2, y_min2, x_max2, y_max2 = box2
    
    inter_x_min = max(x_min1, x_min2)
    inter_y_min = max(y_min1, y_min2)
    inter_x_max = min(x_max1, x_max2)
    inter_y_max = min(y_max1, y_max2)
    
    inter_width = max(0, inter_x_max - inter_x_min)
    inter_height = max(0, inter_y_max - inter_y_min)
    
    intersection_area = inter_width * inter_height
    if (get_intersection_box):
        return intersection_area, [inter_x_min, inter_y_min, inter_x_max, inter_y_max]
        
    return intersection_area

## RFB

In [5]:
all_data_512 = [(image_resizer(tup[0], (512, 512)), object_location_resizer(tup[1], old_dim = (584, 688), new_dim = (512, 512)), tup[2]) for tup in all_data]

In [7]:
class BasicConv(nn.Module):

    def __init__(self, in_planes, out_planes, kernel_size, stride=1, padding=0, dilation=1, groups=1, relu=True, bn=True, bias=False):
        super(BasicConv, self).__init__()
        self.out_channels = out_planes
        self.conv = nn.Conv2d(in_planes, out_planes, kernel_size=kernel_size, stride=stride, padding=padding, dilation=dilation, groups=groups, bias=bias)
        self.bn = nn.BatchNorm2d(out_planes,eps=1e-5, momentum=0.01, affine=True) if bn else None
        self.relu = nn.ReLU(inplace=True) if relu else None

    def forward(self, x):
        x = self.conv(x)
        if self.bn is not None:
            x = self.bn(x)
        if self.relu is not None:
            x = self.relu(x)
        return x


class BasicRFB(nn.Module):

    def __init__(self, in_planes, out_planes, stride=1, scale = 0.1,map_reduce=8):
        super(BasicRFB, self).__init__()
        self.scale = scale
        self.out_channels = out_planes
        inter_planes = in_planes // map_reduce

        self.branch0 = nn.Sequential(
                BasicConv(in_planes, 2*inter_planes, kernel_size=1, stride=stride),
                BasicConv(2*inter_planes, 2*inter_planes, kernel_size=3, stride=1, padding=1,relu=False)
                )
        self.branch1 = nn.Sequential(
                BasicConv(in_planes, inter_planes, kernel_size=1, stride=1),
                BasicConv(inter_planes, 2*inter_planes, kernel_size=(3,3), stride=stride, padding=(1,1)),
                BasicConv(2*inter_planes, 2*inter_planes, kernel_size=3, stride=1, padding=3, dilation=3, relu=False)
                )
        self.branch2 = nn.Sequential(
                BasicConv(in_planes, inter_planes, kernel_size=1, stride=1),
                BasicConv(inter_planes, (inter_planes//2)*3, kernel_size=3, stride=1, padding=1),
                BasicConv((inter_planes//2)*3, 2*inter_planes, kernel_size=3, stride=stride, padding=1),
                BasicConv(2*inter_planes, 2*inter_planes, kernel_size=3, stride=1, padding=5, dilation=5, relu=False)
                )
        self.branch3 = nn.Sequential(
                BasicConv(in_planes, inter_planes, kernel_size=1, stride=1),
                BasicConv(inter_planes, (inter_planes//2)*3, kernel_size=(1,7), stride=1, padding=(0,3)),
                BasicConv((inter_planes//2)*3, 2*inter_planes, kernel_size=(7,1), stride=stride, padding=(3,0)),
                BasicConv(2*inter_planes, 2*inter_planes, kernel_size=3, stride=1, padding=7, dilation=7, relu=False)
                )

        self.ConvLinear = BasicConv(8*inter_planes, out_planes, kernel_size=1, stride=1, relu=False)
        self.shortcut = BasicConv(in_planes, out_planes, kernel_size=1, stride=stride, relu=False)
        self.relu = nn.ReLU(inplace=False)

    def forward(self,x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)

        out = torch.cat((x0,x1,x2,x3),1)
        out = self.ConvLinear(out)
        short = self.shortcut(x)
        out = out*self.scale + short
        out = self.relu(out)

        return out

class BasicRFB_c(nn.Module):

    def __init__(self, in_planes, out_planes, stride=1, scale = 0.1,map_reduce=8):
        super(BasicRFB_c, self).__init__()
        self.scale = scale
        self.out_channels = out_planes
        inter_planes = in_planes // map_reduce
        self.branch0 = nn.Sequential(
                BasicConv(in_planes, 2*inter_planes, kernel_size=1, stride=stride),
                BasicConv(2*inter_planes, 2*inter_planes, kernel_size=3, stride=1, padding=1,relu=False)
                )
        self.branch1 = nn.Sequential(
                BasicConv(in_planes, inter_planes, kernel_size=1, stride=1),
                BasicConv(inter_planes, 2*inter_planes, kernel_size=(3,3), stride=stride, padding=(1,1)),
                BasicConv(2*inter_planes, 2*inter_planes, kernel_size=3, stride=1, padding=3, dilation=3, relu=False)
                )
        self.branch2 = nn.Sequential(
                BasicConv(in_planes, inter_planes, kernel_size=1, stride=1),
                BasicConv(inter_planes, (inter_planes//2)*3, kernel_size=(1,7), stride=1, padding=(0,3)),
                BasicConv((inter_planes//2)*3, 2*inter_planes, kernel_size=(7,1), stride=stride, padding=(3,0)),
                BasicConv(2*inter_planes, 2*inter_planes, kernel_size=3, stride=1, padding=7, dilation=7, relu=False)
                )

        self.ConvLinear = BasicConv(6*inter_planes, out_planes, kernel_size=1, stride=1, relu=False)
        self.shortcut = BasicConv(in_planes, out_planes, kernel_size=1, stride=stride, relu=False)
        self.relu = nn.ReLU(inplace=False)

    def forward(self,x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        x2 = self.branch2(x)

        out = torch.cat((x0,x1,x2),1)
        out = self.ConvLinear(out)
        short = self.shortcut(x)
        out = out*self.scale + short
        out = self.relu(out)

        return out

class BasicRFB_a(nn.Module):

    def __init__(self, in_planes, out_planes, stride=1, scale = 0.1):
        super(BasicRFB_a, self).__init__()
        self.scale = scale
        self.out_channels = out_planes
        inter_planes = in_planes //8


        self.branch0 = nn.Sequential(
                BasicConv(in_planes, inter_planes, kernel_size=1, stride=1),
                BasicConv(inter_planes, inter_planes, kernel_size=3, stride=1, padding=1,relu=False)
                )
        self.branch1 = nn.Sequential(
                BasicConv(in_planes, inter_planes, kernel_size=1, stride=1),
                BasicConv(inter_planes, inter_planes, kernel_size=(3,1), stride=1, padding=(1,0)),
                BasicConv(inter_planes, inter_planes, kernel_size=3, stride=1, padding=3, dilation=3, relu=False)
                )
        self.branch2 = nn.Sequential(
                BasicConv(in_planes, inter_planes, kernel_size=1, stride=1),
                BasicConv(inter_planes, inter_planes, kernel_size=(1,3), stride=stride, padding=(0,1)),
                BasicConv(inter_planes, inter_planes, kernel_size=3, stride=1, padding=3, dilation=3, relu=False)
                )
        self.branch3 = nn.Sequential(
                BasicConv(in_planes, inter_planes, kernel_size=1, stride=1),
                BasicConv(inter_planes, inter_planes, kernel_size=(3,1), stride=1, padding=(1,0)),
                BasicConv(inter_planes, inter_planes, kernel_size=3, stride=1, padding=5, dilation=5, relu=False)
                )
        self.branch4 = nn.Sequential(
                BasicConv(in_planes, inter_planes, kernel_size=1, stride=1),
                BasicConv(inter_planes, inter_planes, kernel_size=(1,3), stride=stride, padding=(0,1)),
                BasicConv(inter_planes, inter_planes, kernel_size=3, stride=1, padding=5, dilation=5, relu=False)
                )
        self.branch5 = nn.Sequential(
                BasicConv(in_planes, inter_planes//2, kernel_size=1, stride=1),
                BasicConv(inter_planes//2, (inter_planes//4)*3, kernel_size=(1,3), stride=1, padding=(0,1)),
                BasicConv((inter_planes//4)*3, inter_planes, kernel_size=(3,1), stride=stride, padding=(1,0)),
                BasicConv(inter_planes, inter_planes, kernel_size=3, stride=1, padding=7, dilation=7, relu=False)
                )

        self.branch6 = nn.Sequential(
                BasicConv(in_planes, inter_planes//2, kernel_size=1, stride=1),
                BasicConv(inter_planes//2, (inter_planes//4)*3, kernel_size=(3,1), stride=1, padding=(1,0)),
                BasicConv((inter_planes//4)*3, inter_planes, kernel_size=(1,3), stride=stride, padding=(0,1)),
                BasicConv(inter_planes, inter_planes, kernel_size=3, stride=1, padding=7, dilation=7, relu=False)
                )
        self.ConvLinear = BasicConv(7*inter_planes, out_planes, kernel_size=1, stride=1, relu=False)
        self.shortcut = BasicConv(in_planes, out_planes, kernel_size=1, stride=stride, relu=False)
        self.relu = nn.ReLU(inplace=False)

    def forward(self,x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        x4 = self.branch4(x)
        x5 = self.branch5(x)
        x6 = self.branch6(x)

        out = torch.cat((x0,x1,x2,x3,x4,x5,x6),1)
        out = self.ConvLinear(out)
        short = self.shortcut(x)
        out = out*self.scale + short
        out = self.relu(out)

        return out

class RFBNet(nn.Module): 

    def __init__(self, img_size, base, extras, head, num_classes):
        print("Entered RFB========")
        super(RFBNet, self).__init__()
        self.num_classes = num_classes

        if img_size == 300:
            self.indicator = 3
        elif img_size == 512:
            self.indicator = 5
        else:
            self.indicator = 5
            print(f"GAK PASS RFBNet__init__ : indicator {self.indicator} Error: Sorry only RFB300 and RFB512 are supported!")
            # return
        self.base = nn.ModuleList(base)
        # Layer learns to scale the l2 normalized features from conv4_3

        self.reduce = BasicConv(512,256,kernel_size=1,stride=1)
        self.up_reduce = BasicConv(1024,256,kernel_size=1,stride=1)

        self.rfb = BasicRFB_a(512, 512, stride=1, scale=1.0)
        self.extras = nn.ModuleList(extras)

        self.loc = nn.ModuleList(head[0])
        self.conf = nn.ModuleList(head[1])


    def forward(self, x, phase=''):
        """Applies network layers and ops on input image(s) x.
        Args:
            x: input image or batch of images. Shape: [batch,3*batch,300,300].
        Return:
            Depending on phase:
            test:
                Variable(tensor) of output class label predictions,
                confidence score, and corresponding location predictions for
                each object detected. Shape: [batch,topk,7]
            train:
                list of concat outputs from:
                    1: confidence layers, Shape: [batch*num_priors,num_classes]
                    2: localization layers, Shape: [batch,num_priors*4]
                    3: priorbox layers, Shape: [2,num_priors*4]
        """
        sources = list()
        loc = list()
        conf = list()

        # apply vgg up to conv4_3 relu
        for k in range(23):
            x = self.base[k](x)

        s1 = self.reduce(x)

        # apply vgg up to fc7
        for k in range(23, len(self.base)):
            x = self.base[k](x)
        s2 = self.up_reduce(x)
        s2 = F.upsample(s2, scale_factor=2, mode='bilinear', align_corners=True)
        s = torch.cat((s1,s2),1) # ERROR HERE

        ss = self.rfb(s)
        sources.append(ss)

        # apply extra layers and cache source layer outputs
        for k, v in enumerate(self.extras):
            x = v(x)
            if k < self.indicator or k%2 ==0:
                sources.append(x)

        # apply multibox head to source layers
        for (x, l, c) in zip(sources, self.loc, self.conf):
            loc.append(l(x).permute(0, 2, 3, 1).contiguous())
            conf.append(c(x).permute(0, 2, 3, 1).contiguous())

        #print([o.size() for o in loc])


        # location predictions
        loc = torch.cat([o.view(o.size(0), -1) for o in loc], 1)
        loc = loc.view(loc.size(0), -1, 4)

        # class confidence scores
        conf = torch.cat([o.view(o.size(0), -1) for o in conf], 1)

        if phase == 'eval' or phase == 'test':
            conf = F.softmax(conf.view(-1, self.num_classes), dim=-1)
        else:
            conf = conf.view(conf.size(0), -1, self.num_classes)

        return (loc, conf)

    def load_model(self, model_file):
        print('Loading model weights from', model_file)
        self.load_state_dict(torch.load(model_file))
        print('Done.')

    def load_backbone(self, model_file):
        print('Loading backbone/base model weights from', model_file)
        self.base.load_state_dict(torch.load(model_file))
        print('Done.')

    def init_weights(self, base=False, rfb=True, extras=True, loc=True, conf=True, reduce=True, up_reduce=True):

        def weights_init(m):
            for key in m.state_dict():
                if key.split('.')[-1] == 'weight':
                    if 'conv' in key:
                        init.kaiming_normal_(m.state_dict()[key], mode='fan_out')
                    if 'bn' in key:
                        m.state_dict()[key][...] = 1
                elif key.split('.')[-1] == 'bias':
                    m.state_dict()[key][...] = 0

        print('Initializing weights...')
        print('Init: base: {}, rfb: {}, extras: {}, loc: {}, conf: {}'.format(base, rfb, extras, loc, conf))
        if base: self.base.apply(weights_init)
        if reduce: self.reduce.apply(weights_init)
        if up_reduce: self.up_reduce.apply(weights_init)
        if rfb: self.rfb.apply(weights_init)
        if extras: self.extras.apply(weights_init)
        if loc: self.loc.apply(weights_init)
        if conf: self.conf.apply(weights_init)
        print('Done.')

# This function is derived from torchvision VGG make_layers()
# https://github.com/pytorch/vision/blob/master/torchvision/models/vgg.py
def vgg(cfg, i, batch_norm=False):
    layers = []
    in_channels = i
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        elif v == 'C':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=True)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    pool5 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
    conv6 = nn.Conv2d(512, 1024, kernel_size=3, padding=6, dilation=6)
    conv7 = nn.Conv2d(1024, 1024, kernel_size=1)
    layers += [pool5, conv6,
               nn.ReLU(inplace=True), conv7, nn.ReLU(inplace=True)]
    return layers

base = {
    '300': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'C', 512, 512, 512, 'M',
            512, 512, 512],
    '512': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'C', 512, 512, 512, 'M',
            512, 512, 512],
}


def add_extras(size, cfg, i, batch_norm=False):
    # Extra layers added to VGG for feature scaling
    layers = []
    in_channels = i
    flag = False
    for k, v in enumerate(cfg):
        if in_channels != 'S':
            if v == 'S':
                if in_channels == 256:
                    layers += [BasicRFB_c(in_channels, cfg[k+1], stride=2, scale = 1.0)]
                else:
                    layers += [BasicRFB(in_channels, cfg[k+1], stride=2, scale = 1.0)]
            else:
                layers += [BasicRFB(in_channels, v, scale = 1.0)]
        in_channels = v
    if size == 512:
        layers += [BasicConv(256,128,kernel_size=1,stride=1)]
        layers += [BasicConv(128,256,kernel_size=4,stride=1,padding=1)]
    elif size ==300:
        layers += [BasicConv(256,128,kernel_size=1,stride=1)]
        layers += [BasicConv(128,256,kernel_size=3,stride=1)]
        layers += [BasicConv(256,128,kernel_size=1,stride=1)]
        layers += [BasicConv(128,256,kernel_size=3,stride=1)]
    else:
        print(f"GAK PASS : size = {size} add_extras: Error: Sorry only RFB300 and RFB512 are supported!")
        layers += [BasicConv(256,128,kernel_size=1,stride=1)]
        layers += [BasicConv(128,256,kernel_size=3,stride=1)]
        layers += [BasicConv(256,128,kernel_size=1,stride=1)]
    return layers

extras = {
    '300': [1024, 'S', 512, 'S', 256],
    '512': [1024, 'S', 512, 'S', 256, 'S', 256,'S',256],
}


def multibox(size, vgg, extra_layers, cfg, num_classes):
    loc_layers = []
    conf_layers = []
    vgg_source = [-2]
    for k, v in enumerate(vgg_source):
        if k == 0:
            loc_layers += [nn.Conv2d(512,
                                 cfg[k] * 4, kernel_size=3, padding=1)]
            conf_layers +=[nn.Conv2d(512,
                                 cfg[k] * num_classes, kernel_size=3, padding=1)]
        else:
            loc_layers += [nn.Conv2d(vgg[v].out_channels,
                                 cfg[k] * 4, kernel_size=3, padding=1)]
            conf_layers += [nn.Conv2d(vgg[v].out_channels,
                        cfg[k] * num_classes, kernel_size=3, padding=1)]
    i = 1
    indicator = 0
    if size == 300:
        indicator = 3
    elif size == 512:
        indicator = 5
    else:
        indicator = 5
        print(f"GAK PASS indicator {indicator}: multibox: Error: Sorry only RFB300 and RFB512 are supported!")
        # return

    for k, v in enumerate(extra_layers):
        if k < indicator or k%2== 0:
            loc_layers += [nn.Conv2d(v.out_channels, cfg[i]
                                 * 4, kernel_size=3, padding=1)]
            conf_layers += [nn.Conv2d(v.out_channels, cfg[i]
                                  * num_classes, kernel_size=3, padding=1)]
            i +=1
    return vgg, extra_layers, (loc_layers, conf_layers)

mbox = {
    '300': [6, 6, 6, 6, 4, 4],  # number of boxes per feature map location
    '512': [6, 6, 6, 6, 6, 4, 4],
}


def build_net(img_size=512, num_classes=2):
    if img_size not in [300, 512]:
        print(f"GAK PASS : size = {img_size} Error: only RFBE300 and RFBE512 are supported!")
        # return

    if img_size not in base.keys():
      base[str(img_size)] = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'C', 512, 512, 512, 'M',
            512, 512, 512]

    if img_size not in extras.keys():
      extras[str(img_size)] = [1024, 'S', 512, 'S', 256, 'S', 256,'S',256]

    if img_size not in mbox.keys():
      mbox[str(img_size)] = [6, 6, 6, 6, 6, 4, 4]
      
    # print(f"base = {base}")
    # print()
    # print(f"extras = {extras}")
    # print('\nNetwork: RFBE_VGG. Image size: %d. Number of classes: %d' % (img_size, num_classes))

    return RFBNet(img_size, *multibox(img_size, vgg(base[str(img_size)], 3),
                                add_extras(img_size, extras[str(img_size)], 1024),
                                mbox[str(img_size)], num_classes), num_classes)

In [19]:
def jaccard(box_a, box_b):
    """Compute the jaccard overlap (IoU) between two sets of boxes.
    The jaccard overlap is simply the intersection over union of two boxes.
    
    Args:
        box_a: (tensor) Ground truth bounding boxes, Shape: [num_objects, 4].
        box_b: (tensor) Prior boxes from priorbox layers, Shape: [num_priors, 4].
    
    Return:
        IoU: (tensor) Shape: [num_objects, num_priors]
    """
    # Intersection box: find the max of the min coords and the min of the max coords
    inter_xmin = torch.max(box_a[:, 0].unsqueeze(1), box_b[:, 0])
    inter_ymin = torch.max(box_a[:, 1].unsqueeze(1), box_b[:, 1])
    inter_xmax = torch.min(box_a[:, 2].unsqueeze(1), box_b[:, 2])
    inter_ymax = torch.min(box_a[:, 3].unsqueeze(1), box_b[:, 3])

    # Compute the width and height of the intersection box
    inter_w = torch.clamp(inter_xmax - inter_xmin, min=0)
    inter_h = torch.clamp(inter_ymax - inter_ymin, min=0)

    # Intersection area
    inter_area = inter_w * inter_h

    # Area of both the boxes
    area_a = (box_a[:, 2] - box_a[:, 0]) * (box_a[:, 3] - box_a[:, 1])  # [num_objects]
    area_b = (box_b[:, 2] - box_b[:, 0]) * (box_b[:, 3] - box_b[:, 1])  # [num_priors]

    # Union area
    union_area = area_a.unsqueeze(1) + area_b - inter_area

    # IoU calculation
    return inter_area / union_area  # [num_objects, num_priors]

def point_form(boxes):
    """ Convert prior boxes from (cx, cy, w, h) to (xmin, ymin, xmax, ymax).
    Args:
        boxes: (tensor) prior boxes in center-offset form (cx, cy, w, h).
                Shape: [num_priors, 4]
    Return:
        boxes: (tensor) Converted boxes in point (corner) form (xmin, ymin, xmax, ymax).
                Shape: [num_priors, 4]
    """
    return torch.cat((boxes[:, :2] - boxes[:, 2:] / 2,  # xmin, ymin
                      boxes[:, :2] + boxes[:, 2:] / 2), 1)  # xmax, ymax


############################################
# from lib.utils.box_utils import match, log_sum_exp
def log_sum_exp(x):
    """Utility function for computing log_sum_exp while determining
    This will be used to determine unaveraged confidence loss across
    all examples in a batch.
    Args:
        x (Variable(tensor)): conf_preds from conf layers
    """
    x_max = x.data.max()
    return torch.log(torch.sum(torch.exp(x-x_max), 1, keepdim=True)) + x_max
def match(threshold, truths, priors, variances, labels, loc_t, conf_t, idx):
    """Match each prior box with the ground truth box of the highest jaccard
    overlap, encode the bounding boxes, then return the matched indices
    corresponding to both confidence and location preds.
    Args:
        threshold: (float) The overlap threshold used when mathing boxes.
        truths: (tensor) Ground truth boxes, Shape: [num_obj, num_priors].
        priors: (tensor) Prior boxes from priorbox layers, Shape: [n_priors,4].
        variances: (tensor) Variances corresponding to each prior coord,
            Shape: [num_priors, 4].
        labels: (tensor) All the class labels for the image, Shape: [num_obj].
        loc_t: (tensor) Tensor to be filled w/ endcoded location targets.
        conf_t: (tensor) Tensor to be filled w/ matched indices for conf preds.
        idx: (int) current batch index
    Return:
        The matched indices corresponding to 1)location and 2)confidence preds.
    """
    # jaccard index
    overlaps = jaccard(
        truths,
        point_form(priors)
    )
    # (Bipartite Matching)
    # [1,num_objects] best prior for each ground truth
    best_prior_overlap, best_prior_idx = overlaps.max(1, keepdim=True)
    # [1,num_priors] best ground truth for each prior
    best_truth_overlap, best_truth_idx = overlaps.max(0, keepdim=True)
    best_truth_idx.squeeze_(0)
    best_truth_overlap.squeeze_(0)
    best_prior_idx.squeeze_(1)
    best_prior_overlap.squeeze_(1)
    best_truth_overlap.index_fill_(0, best_prior_idx, 2)  # ensure best prior
    # TODO refactor: index  best_prior_idx with long tensor
    # ensure every gt matches with its prior of max overlap
    for j in range(best_prior_idx.size(0)):
        best_truth_idx[best_prior_idx[j]] = j
    matches = truths[best_truth_idx]          # Shape: [num_priors,4]
    conf = labels[best_truth_idx]          # Shape: [num_priors]
    conf[best_truth_overlap < threshold] = 0  # label as background
    loc = encode(matches, priors, variances)
    loc_t[idx] = loc    # [num_priors,4] encoded offsets to learn
    conf_t[idx] = conf  # [num_priors] top class label for each prior

def encode(matched, priors, variances):
    """Encode the variances from the priorbox layers into the ground truth boxes
    we have matched (based on jaccard overlap) with the prior boxes.
    Args:
        matched: (tensor) Coords of ground truth for each prior in point-form
            Shape: [num_priors, 4].
        priors: (tensor) Prior boxes in center-offset form
            Shape: [num_priors,4].
        variances: (list[float]) Variances of priorboxes
    Return:
        encoded boxes (tensor), Shape: [num_priors, 4]
    """

    # dist b/t match center and prior's center
    g_cxcy = (matched[:, :2] + matched[:, 2:])/2 - priors[:, :2]
    # encode variance
    g_cxcy /= (variances[0] * priors[:, 2:])
    # match wh / prior wh
    g_wh = (matched[:, 2:] - matched[:, :2]) / priors[:, 2:]
    g_wh = torch.log(g_wh) / variances[1]

    # return target for smooth_l1_loss
    return torch.cat([g_cxcy, g_wh], 1)  # [num_priors,4]
############################################

class MultiBoxLoss(nn.Module):
    """SSD Weighted Loss Function
    Compute Targets:
        1) Produce Confidence Target Indices by matching  ground truth boxes
           with (default) 'priorboxes' that have jaccard index > threshold parameter
           (default threshold: 0.5).
        2) Produce localization target by 'encoding' variance into offsets of ground
           truth boxes and their matched  'priorboxes'.
        3) Hard negative mining to filter the excessive number of negative examples
           that comes with using a large number of default bounding boxes.
           (default negative:positive ratio 3:1)
    Objective Loss:
        L(x,c,l,g) = (Lconf(x, c) + αLloc(x,l,g)) / N
        Where, Lconf is the CrossEntropy Loss and Lloc is the SmoothL1 Loss
        weighted by α which is set to 1 by cross val.
        Args:
            c: class confidences,
            l: predicted boxes,
            g: ground truth boxes
            N: number of matched default boxes
        See: https://arxiv.org/pdf/1512.02325.pdf for more details.
    """


    def __init__(self, num_classes, overlap_thresh, negpos_ratio):
        super(MultiBoxLoss, self).__init__()

        self.num_classes = num_classes
        self.threshold = overlap_thresh
        self.negpos_ratio = negpos_ratio
        self.variance = [0.1, 0.2]

    # TODO: update losses: https://pytorch.org/docs/stable/nn.html?highlight=f%20cross_entropy#torch.nn.functional.cross_entropy
    def forward(self, predictions, priors, targets):
        """Multibox Loss
        Args:
            predictions (tuple): A tuple containing loc preds, conf preds,
            and prior boxes from SSD net.
                conf shape: torch.size(batch_size,num_priors,num_classes)
                loc shape: torch.size(batch_size,num_priors,4)
                priors shape: torch.size(num_priors,4)

            ground_truth (tensor): Ground truth boxes and labels for a batch,
                shape: [batch_size,num_objs,5] (last idx is the label).
        """

        # print("INSIDE MultiBoxLoss forward()")
        # print(f"type(predictions) = {type(predictions)}")       #  <class 'tuple'>
        # print(f"len(predictions) = {len(predictions)}")         # 2
        # print(f"predictions[0].shape = {predictions[0].shape}") # torch.Size([1, 40376, 4])
        # # print(f"predictions = {predictions}")
        # print()
        # print(f"type(priors) = {type(priors)}")                # <class 'torch.Tensor'>
        # print(f"priors.shape = {priors.shape}")                # torch.Size([32756, 4])
        # # print(f"priors = {priors}")
        # print()
        # print(f"type(targets) = {type(targets)}")              # <class 'list'>
        # print(f"len(targets) = {len(targets)}")                # 1
        # print(f"targets[0].shape = {targets[0].shape}")        # torch.Size([4, 5])
        # print(f"targets = {targets}")

        loc_data, conf_data = predictions
        priors = priors
        num = loc_data.size(0)
        num_priors = (priors.size(0))

        # match priors (default boxes) and ground truth boxes
        loc_t = torch.Tensor(num, num_priors, 4)
        conf_t = torch.LongTensor(num, num_priors)
        for idx in range(num):
            truths = targets[idx][:,:-1].data
            labels = targets[idx][:,-1].data
            defaults = priors.data
            match(self.threshold, truths, defaults, self.variance, labels, loc_t, conf_t, idx)


        ## GAK WAS HERE 
        ## 01
        # loc_t = loc_t.cuda()
        # conf_t = conf_t.cuda()
        ## / 01
        ## 02
        if torch.cuda.is_available():
            device = torch.device("cuda")
        else:
            device = torch.device("cpu")
        loc_t = loc_t.to(device)
        conf_t = conf_t.to(device)
        ## / 02

        pos = conf_t > 0

        # Localization Loss (Smooth L1)
        # Shape: [batch,num_priors,4]
        pos_idx = pos.unsqueeze(pos.dim()).expand_as(loc_data)
        loc_p = loc_data[pos_idx].view(-1,4)
        loc_t = loc_t[pos_idx].view(-1,4)
        loss_l = F.smooth_l1_loss(loc_p, loc_t, reduction='sum')

        # Compute max conf across batch for hard negative mining
        batch_conf = conf_data.view(-1, self.num_classes)
        loss_c = log_sum_exp(batch_conf) - batch_conf.gather(1, conf_t.view(-1,1))

        # Hard Negative Mining
        loss_c[pos.view(-1,1)] = 0 # filter out pos boxes for now
        loss_c = loss_c.view(num, -1)
        _,loss_idx = loss_c.sort(1, descending=True)
        _,idx_rank = loss_idx.sort(1)
        num_pos = pos.long().sum(1,keepdim=True)
        num_neg = torch.clamp(self.negpos_ratio*num_pos, max=pos.size(1)-1)
        neg = idx_rank < num_neg.expand_as(idx_rank)

        # Confidence Loss Including Positive and Negative Examples
        pos_idx = pos.unsqueeze(2).expand_as(conf_data)
        neg_idx = neg.unsqueeze(2).expand_as(conf_data)
        conf_p = conf_data[(pos_idx+neg_idx).gt(0)].view(-1,self.num_classes)
        targets_weighted = conf_t[(pos+neg).gt(0)]
        loss_c = F.cross_entropy(conf_p, targets_weighted, reduction='sum')

        # Sum of losses: L(x,c,l,g) = (Lconf(x, c) + αLloc(x,l,g)) / N

        N = max(num_pos.sum().float(), 1.)
        loss_l /= N
        loss_c /= N

        return loss_l, loss_c


In [10]:
anchorbox_params={
    "RFB_512" : {
        'feature_maps' : [64, 32, 16, 8, 4, 2, 1], # @@@@@@@@@@@@@@@@@@

        'min_dim' : 512,

        'steps' : [8, 16, 32, 64, 128, 256, 512],

        'min_sizes' : [20.48, 51.2, 133.12, 215.04, 296.96, 378.88, 460.8],

        'max_sizes' : [51.2, 133.12, 215.04, 296.96, 378.88, 460.8, 542.72],

        'aspect_ratios' : [[2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2], [2]],

        'variance' : [0.1, 0.2],

        'clip' : True,
    },


    # VOC_300
    # RFB VGG for 300x300
    "RFB_300" : {
        'feature_maps' : [38, 19, 10, 5, 3, 1],

        'min_dim' : 300,

        'steps' : [8, 16, 32, 64, 100, 300],

        'min_sizes' : [30, 60, 111, 162, 213, 264],

        'max_sizes' : [60, 111, 162, 213, 264, 315],

        'aspect_ratios' : [[2, 3], [2, 3], [2, 3], [2, 3], [2], [2]],

        'variance' : [0.1, 0.2],

        'clip' : True,
    },
}

class Anchors_W(object):
    """Compute anchor box coordinates in center-offset form for each source feature map.
    """
    def __init__(self, cfg):
        self.image_size = cfg['min_dim']                     # 512
        self.variance = cfg['variance'] or [0.1, 0.2]        # [0.1, 0.2]
        self.feature_maps = cfg['feature_maps']              # [64, 32, 16, 8, 4, 2, 1]
        self.min_sizes = cfg['min_sizes']                    # [20.48, 51.2, 133.12, 215.04, 296.96, 378.88, 460.8]
        self.max_sizes = cfg['max_sizes']                    # [51.2, 133.12, 215.04, 296.96, 378.88, 460.8, 542.72]
        self.steps = cfg['steps']                            # [8, 16, 32, 64, 128, 256, 512]
        self.aspect_ratios = cfg['aspect_ratios']            # [[2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2], [2]]
        self.clip = cfg['clip']                              # True

        for v in self.variance:
            if v <= 0:
                raise ValueError('Variances must be greater than 0')

    def forward(self):
        """Generate the anchor boxes for each feature map."""
        anchor_boxes = []

        for k, f in enumerate(self.feature_maps):  # Iterate through feature maps
            for i, j in product(range(f), repeat=2):  # Iterate through grid positions
                f_k = self.image_size / self.steps[k]  # e.g., 512 / [8, 16, 32, ...] -> [64.0, 32.0, ...]
                cx = (j + 0.5) / f_k  # Center x
                cy = (i + 0.5) / f_k  # Center y

                # Create anchor for aspect ratio 1
                s_k = self.min_sizes[k] / self.image_size  # Scale relative to image size
                anchor_boxes += [cx, cy, s_k, s_k]

                # Create an additional anchor box for aspect ratio 1 with larger size
                s_k_prime = sqrt(s_k * (self.max_sizes[k] / self.image_size))
                anchor_boxes += [cx, cy, s_k_prime, s_k_prime]

                # Create anchors for other aspect ratios
                for ar in self.aspect_ratios[k]:
                    anchor_boxes += [cx, cy, s_k * sqrt(ar), s_k / sqrt(ar)]
                    anchor_boxes += [cx, cy, s_k / sqrt(ar), s_k * sqrt(ar)]

        # Convert the list of anchor boxes into a tensor
        output = torch.Tensor(anchor_boxes).view(-1, 4)

        # Clip the boxes to ensure coordinates are within [0, 1] range
        if self.clip:
            output.clamp_(max=1., min=0.)

        return output


In [17]:
# Updated voc_eval function
def voc_eval(detections, annotations, cid, ovthresh=0.5):
    """
    Top-level function that does the PASCAL VOC evaluation.
    Args:
        detections: dict of detections for a specific class (keyed on image IDs), where
                    each entry is a list of detection bounding boxes and confidences.
        annotations: dict of ground truth annotations keyed on image IDs, where each entry
                    is a list of ground truth boxes with their respective class IDs.
        cid: Class ID to evaluate.
        ovthresh: Overlap threshold for considering a detection as True Positive.
    Returns:
        rec: Recall for the class
        prec: Precision for the class
        ap: Average Precision (AP) for the class
    """
    use_07_metric = False  # Use 11-point interpolation for PASCAL VOC 2007

    # Extract gt objects for this class
    class_recs = {}
    npos = 0
    for imagename in annotations:
        # For each image, list of objects: [[x1, y1, x2, y2, class_id], ...]
        R = [obj[:4] for obj in annotations[imagename] if int(obj[-1]) == cid]
        bbox = np.array(R)
        difficult = np.array([0] * len(R)).astype(np.bool)  # No difficult examples
        det = [False] * len(R)
        npos += sum(~difficult)
        class_recs[imagename] = {'bbox': bbox, 'difficult': difficult, 'det': det}

    # Detections for this class
    image_ids = []
    confidence = []
    BB = []
    class_dict = detections.get(cid, {})
    for image_file in class_dict:
        dets = class_dict[image_file]
        for k in range(dets.shape[0]):
            image_ids.append(image_file)
            BB.append(dets[k, 0:4])
            confidence.append(dets[k, -1])

    BB = np.array(BB)
    confidence = np.array(confidence)

    nd = len(image_ids)
    tp = np.zeros(nd)
    fp = np.zeros(nd)
    if npos == 0:  # Handle case where there are no ground truths for this class
        return np.zeros(0), np.zeros(0), 0.0
    if BB.shape[0] > 0:
        # Sort by confidence
        sorted_ind = np.argsort(-confidence)
        BB = BB[sorted_ind, :]
        image_ids = [image_ids[x] for x in sorted_ind]

        # Go through detections and mark TPs and FPs
        for d in range(nd):
            # Check if the image ID exists in the ground truth for this class
            if image_ids[d] not in class_recs:
                fp[d] = 1.  # Count it as a false positive since no ground truth exists for this image
                continue

            R = class_recs[image_ids[d]]
            bb = BB[d, :].astype(float)
            ovmax = -np.inf
            BBGT = R['bbox'].astype(float)

            if BBGT.size > 0:
                # Compute IoU overlaps
                ixmin = np.maximum(BBGT[:, 0], bb[0])
                iymin = np.maximum(BBGT[:, 1], bb[1])
                ixmax = np.minimum(BBGT[:, 2], bb[2])
                iymax = np.minimum(BBGT[:, 3], bb[3])
                iw = np.maximum(ixmax - ixmin + 1., 0.)
                ih = np.maximum(iymax - iymin + 1., 0.)
                inters = iw * ih
                uni = ((bb[2] - bb[0] + 1.) * (bb[3] - bb[1] + 1.) + 
                       (BBGT[:, 2] - BBGT[:, 0] + 1.) * (BBGT[:, 3] - BBGT[:, 1] + 1.) - inters)
                overlaps = inters / uni
                ovmax = np.max(overlaps)
                jmax = np.argmax(overlaps)

            if ovmax > ovthresh:
                if not R['difficult'][jmax]:
                    if not R['det'][jmax]:
                        tp[d] = 1.
                        R['det'][jmax] = 1
                    else:
                        fp[d] = 1.
            else:
                fp[d] = 1.

    # Compute precision-recall
    fp = np.cumsum(fp)
    tp = np.cumsum(tp)
    rec = tp / float(npos)
    prec = tp / np.maximum(tp + fp, np.finfo(np.float64).eps)
    ap = voc_ap(rec, prec, use_07_metric)

    return rec, prec, ap


# Helper function to compute AP
def voc_ap(rec, prec, use_07_metric=False):
    """ Compute the average precision (AP) using either the 07 or 12/10 metric """
    if use_07_metric:
        ap = 0.
        for t in np.arange(0., 1.1, 0.1):
            if np.sum(rec >= t) == 0:
                p = 0
            else:
                p = np.max(prec[rec >= t])
            ap += p / 11.
    else:
        mrec = np.concatenate(([0.], rec, [1.]))
        mpre = np.concatenate(([0.], prec, [0.]))

        for i in range(mpre.size - 1, 0, -1):
            mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

        i = np.where(mrec[1:] != mrec[:-1])[0]
        ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])

    return ap

# Evaluation function for all classes
def evaluate_detections(detections_dict, annotations, classes, iou_thresh=0.5):
    aps = []
    print('--------------------------------')
    print('Class : \t AP (100)')
    print('--------------------------------')

    for cid, classname in enumerate(classes):
        if classname == '__background__':
            continue
        # print(f"$$$$ voc_eval detections_dict= {detections_dict} <> annotations = {annotations} <>  cid = {cid} <> iou_thresh = {iou_thresh}" )       
        rec, prec, ap = voc_eval(detections_dict, annotations, cid, iou_thresh)
        # print(f"@@@@ voc_eval")  
        aps.append(ap)
        print(f'{classname} : \t{100 * ap:.2f}')

    mAP = 100 * np.mean(aps)
    print('--------------------------------')
    print(f'mAP: \t{mAP:.2f}')
    print('--------------------------------')

    return aps, mAP


## gak was here asd


# # Print mAP
# print(f'Epoch {epoch+1}, mAP: {mAP:.2f}')

# # Step the scheduler at the end of each epoch to adjust learning rate
# scheduler.step()

In [12]:
anchor_cfg = anchorbox_params["RFB_512"]
anchors_w = Anchors_W(anchor_cfg)
anchors = anchors_w.forward()  ## qwe

In [20]:
import torch
import numpy as np

# Assuming img_size=512 and num_classes=5 as per your dataset
img_size = 512
num_classes = 5
batch_size = 2
num_epochs = 1

# Split data into training and validation sets
val_percent = 0.2
len_all_data_512 = 10
val_start_index = len_all_data_512 - int(len_all_data_512*val_percent)

train_data = all_data_512[:val_start_index]  # Example: First 8 images for training
val_data = all_data_512[val_start_index:10]    # Example: Last 2 images for validation

# Instantiate the RFBNet model
this_rfb = RFBNet(img_size, *multibox(img_size, vgg(base[str(img_size)], 3),
                                add_extras(img_size, extras[str(img_size)], 1024),
                                mbox[str(img_size)], num_classes), num_classes)

# Set model to training mode
this_rfb.train()

# Instantiate the MultiBoxLoss
multibox_loss = MultiBoxLoss(num_classes=num_classes, overlap_thresh=0.5, negpos_ratio=3)

# Optimizer
optimizer = torch.optim.Adam(this_rfb.parameters(), lr=0.001)

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Generate anchor boxes using Anchors_W class
anchor_cfg = anchorbox_params["RFB_512"]
anchors_w = Anchors_W(anchor_cfg)
priors = anchors_w.forward()

# Move model and priors to the correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
this_rfb.to(device)
priors = priors.to(device)

### TRAINING AND VALIDATION LOOP ###
for epoch in range(num_epochs):
    ### TRAINING ###
    this_rfb.train()  # Set the model to training mode
    for i in range(0, len(train_data), batch_size):
        # Gather batch of images, bounding boxes, and class labels from training data
        batch_images = [train_data[j][0] for j in range(i, min(i + batch_size, len(train_data)))]
        batch_boxes = [train_data[j][1] for j in range(i, min(i + batch_size, len(train_data)))]
        batch_labels = [train_data[j][2] for j in range(i, min(i + batch_size, len(train_data)))]
        
        # Stack the batch of images into a single tensor
        batch_images = torch.stack(batch_images).to(device)

        # Forward pass through the model to get predictions (loc_preds, conf_preds)
        loc_preds, conf_preds = this_rfb(batch_images)

        # Prepare ground truth targets (bounding boxes + labels)
        targets = [torch.cat([boxes, labels.unsqueeze(1)], dim=1).to(device) for boxes, labels in zip(batch_boxes, batch_labels)]

        # Compute the loss
        loss_l, loss_c = multibox_loss((loc_preds, conf_preds), priors, targets)

        # Backpropagation
        optimizer.zero_grad()
        total_loss = loss_l + loss_c
        total_loss.backward()
        optimizer.step()

    ### VALIDATION ###
    this_rfb.eval()  # Set the model to evaluation mode
    val_loc_loss = 0.0
    val_conf_loss = 0.0

    # Create dictionary to store detections for evaluation
    detections_dict = {cid: {} for cid in range(num_classes)}  # cid corresponds to class ID
    with torch.no_grad():  # Disable gradient calculation during validation
        for i in range(0, len(val_data), batch_size):
            # Gather batch of images, bounding boxes, and class labels from validation data
            val_images = [val_data[j][0] for j in range(i, min(i + batch_size, len(val_data)))]
            val_boxes = [val_data[j][1] for j in range(i, min(i + batch_size, len(val_data)))]
            val_labels = [val_data[j][2] for j in range(i, min(i + batch_size, len(val_data)))]
            
            # Stack the batch of validation images into a single tensor
            val_images = torch.stack(val_images).to(device)

            # Forward pass through the model to get predictions (loc_preds, conf_preds)
            val_loc_preds, val_conf_preds = this_rfb(val_images)

            # Store predictions for evaluation later
            for idx, img_id in enumerate(range(i, min(i + batch_size, len(val_data)))):
                # Convert predictions into usable format
                decoded_boxes = val_loc_preds[idx].cpu().detach().numpy()
                confidences = val_conf_preds[idx].cpu().detach().numpy()
                for cid in range(1, num_classes):  # Skipping background class (cid=0)
                    class_mask = confidences[:, cid] > 0.5  # Apply threshold (0.5 example)
                    if class_mask.sum() > 0:
                        detections_dict[cid][img_id] = np.hstack([decoded_boxes[class_mask], confidences[class_mask, cid].reshape(-1, 1)])

            # Prepare ground truth targets for validation
            val_targets = [torch.cat([boxes, labels.unsqueeze(1)], dim=1).to(device) for boxes, labels in zip(val_boxes, val_labels)]

            # Compute the loss on the validation set
            val_loss_l, val_loss_c = multibox_loss((val_loc_preds, val_conf_preds), priors, val_targets)
            val_loc_loss += val_loss_l.item()
            val_conf_loss += val_loss_c.item()

    # Average validation loss per image
    avg_val_loc_loss = val_loc_loss / len(val_data)
    avg_val_conf_loss = val_conf_loss / len(val_data)

    # Print the losses for the current epoch
    print(f'Epoch {epoch+1}, Train Loc Loss: {loss_l.item()}, Train Conf Loss: {loss_c.item()}')
    print(f'Epoch {epoch+1}, Val Loc Loss: {avg_val_loc_loss}, Val Conf Loss: {avg_val_conf_loss}')

Entered RFB========


C:\Users\Gaura\AppData\Local\Temp\ipykernel_18808\1391546839.py:232: UserWarning: `nn.functional.upsample` is deprecated. Use `nn.functional.interpolate` instead.
  s2 = F.upsample(s2, scale_factor=2, mode='bilinear', align_corners=True)


Epoch 1, Train Loc Loss: 159250.46875, Train Conf Loss: 12.415828704833984
Epoch 1, Val Loc Loss: 40913.890625, Val Conf Loss: 3.415937900543213


In [21]:
annotations = {img_id: [all_data_512[img_id][1].cpu().numpy()] for img_id in range(val_start_index, val_start_index+len(val_data))}
corrected_annotations = {}
# print(f"before correction annotations: {annotations}")
this_val_id = -1
for k, v in annotations.items():
    this_val_id += 1
    final_list = []
    true_ids = val_data[this_val_id][2].tolist()
    # print(true_ids)
    for tri, bb in zip(true_ids, v[0].tolist()):
        bb.append(tri)
        final_list.append(bb)
    corrected_annotations[k] = final_list
annotations = corrected_annotations

# {0: 'gun', 1: 'knife', 2: 'plier', 3: 'scissor', 4: 'wrench'}

aps, mAP = evaluate_detections(detections_dict, annotations, classes=['gun', 'knife', 'plier', 'scissor', 'wrench',  '__background__'])
# Print mAP
print(f'Epoch {epoch+1}, mAP: {mAP:.2f}')

# Step the scheduler at the end of each epoch to adjust learning rate

--------------------------------
Class : 	 AP (100)
--------------------------------
gun : 	0.00
knife : 	0.00
plier : 	0.00
scissor : 	0.00
wrench : 	0.00
--------------------------------
mAP: 	0.00
--------------------------------
Epoch 1, mAP: 0.00
